In [1]:
%matplotlib inline
%config InlineBackend.figure_format='retina'
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from astropy.coordinates import SkyCoord
from astropy.io import fits
from astropy.wcs import WCS
from scipy.spatial import cKDTree
from matplotlib import rc
rc('font', **{'family': 'serif', 'serif': ['Computer Modern Roman'], 'size':14})
rc('text', usetex=True)

In [61]:
def make_gloess_in(mapnum, filters):
    g_outs = []
    for filt in filters:
        dates = pd.read_csv('dateslist.txt', names=['filt', 'mapnum', 'epoch', 'mjd'], delim_whitespace=True)
        dates = dates[(dates.filt == filt) & (dates.mapnum == mapnum)].sort(columns='epoch')
        names = ['id', 'x', 'y']#, 'm0', 'e0', 'm1', 'e1']
        for i in range(14):
            names.append('m{}'.format(i))
            names.append('e{}'.format(i))
        df = pd.read_csv('omegaCen_{}_{}um_rrl.raw'.format(mapnum, filt),
                        delim_whitespace=True, names=names)
        g = pd.read_csv('Kaluzny2004rrl.txt', delim_whitespace=True)
        g_out = np.zeros((12,25))
        g_out[:,0] = dates.mjd.values
        g_out[:,2::2] += 9.9999
        g_out[:,1::2] += 99.999
        for i in range(len(df.id.values)):
            starid = g.id[g.id == df.id.values[i]].values.astype(int)[0]
            per = g.per[g.id == df.id.values[i]].values.astype(float)[0]
            stuff = df[df.index==i]
            mags = stuff[['m{}'.format(j) for j in range(1,13)]]
            merrs = stuff[['e{}'.format(j) for j in range(1,13)]]
            if filt == '3p6':
                g_out[:,17] = mags
                g_out[:,18] = merrs
            else:
                g_out[:,19] = mags
                g_out[:,20] = merrs
            f = open('{}_{}.gloess_in'.format(starid, filt), 'w')
            f.write('{}\n'.format(starid))
            f.write('{}\n'.format(per))
            f.write('12\n')
            f.write('0.10 0.10 0.10 0.10 0.10 0.10 0.10 0.10 0.10 0.10 0.10 0.10\n')
            for k in g_out:
                f.write(' '.join(map(str, k)) + '\n')
            f.close()

In [64]:
make_gloess_in(3, ['3p6','4p5'])

In [ ]:
import shutil
for i in glob.glob('maybe_not_crap/*.eps'):
    shutil.copy('fits/' + i.split('/')[-1].split('.')[0] + '.glo_avs', 'maybe_not_crap/')
    shutil.copy('fits/' + i.split('/')[-1].split('.')[0] + '.glo_fits', 'maybe_not_crap/')